# ADU Histogram Plotter
This notebook will be useful for setting the ADU cutoff. ADU is the unit which the Jungfrau4M detector outputs, and is roughly in keV. Fluoresence may obscure any nonlinear effects as seen in LV04, so making sure we get this right at the beginning will be helpful. Modified from Ian. From past experience, it is not good to set an upper bound as it will cause a deviation from the theory at low q, since double photon per pixel events are more likely there than at large q. Also don't set the upper bound too high, since the inelastic signal will be at a range below the input photon energy. Usually a couple hundred eV. Use `xrayscatteringtools.theory.iam.iam_compton_spectrum` to get an idea of how far this spectrum extends downward. Setting a lower bound for the ADU cutoff too high will cause deviations at high q, where split-pixel photon detections are too high.

Some common values:
 - SF$_6 \to$ 2.5 keV.
 - Ne $\to$ 1 keV.
## First loading in modules.

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import psana
import xrayscatteringtools as xrst
from tqdm.auto import tqdm

## Using the psana library to make a datasource object for the specific run

In [ ]:
##########################
runNumber = 15 # A single run number goes here.
experiment = xrst.get_config('expNumber')[0] # Auto load from config.yaml
##########################
dsname = 'exp=%s:run=%d:smd' % (experiment , runNumber) 
ds = psana.DataSource(dsname)
events = ds.events()

## Setting up the Jungfrau4M detector object and pulling off images, then compute the histogram

In [ ]:
###############
n_images = 10 # Amount of images we want to pull from the run, 
###############
images = np.full((8,512,1024,n_images),np.nan) # Preallocation of the image array
jf = psana.Detector('jungfrau4M') # Grabbing the detector
# Grabbing the mask from the first event, this is the same mask which will be applied to the smalldata azav function. Equivalent to cmask key in h5 files
cmask = jf.mask(
    runNumber,
    unbond=True,
    unbondnbrs=True,
    status=True,
    edges=True,
    central=True,
    calib=True,
).astype(bool)
# Grabiing the images
evts = tqdm(enumerate(ds.events()), total=n_images)
for _idx, evt in evts:
    if _idx==n_images: # Breaking out of the loop if we reach n_images
        evts.close()
        break
    images[:,:,:,_idx][cmask] = jf.calib(evt,cmpars=(7, 0, 10),mbits=0)[cmask] # Grabbing pedestal-subtracted, non-masked image and appending it to the array
print(images.shape)
hists,bins = np.histogram(images.flatten(),bins=np.linspace(0,50,200)) # Binning the flattened J4M images into the histogram variables

## Plotting the histogram

In [ ]:
##########
cutoff = 5 # Input cutoff in keV here
sample = xrst.get_config_for_runs(runNumber,"samples","sample") # Auto from config.yaml
##########
plt.figure(figsize=(8,5))
plt.semilogy(bins[:-1],hists)
plt.xlabel('Pixel Value (ADU)')
plt.ylabel('Counts')
plt.axvline(cutoff,linestyle='--',label=f'{cutoff} keV',color='r') # You can add vertical lines here with this line
plt.legend(fontsize=15)
plt.title(f'{experiment}: Run {runNumber}, Sample: {sample}')
plt.show()